<a href="https://colab.research.google.com/drive/1rDgf0nh-u8cmKL3wCs7nHsxqxi8HyRgu" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tifffile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import pandas as pd
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
import matplotlib
import matplotlib.pyplot as plt
import pdb

In [ ]:
## Functions

## Calculate scaling in x and y for vector to raster transformation
def getScaleXY(imgSize, xMax, yMin):
    h, w = imgSize
    wc = w * (w / (w + 1))
    hc = h * (h / (h + 1))
    return wc / xMax, hc / yMin


In [ ]:
## Intensity normalize images 
def scaleImg(imgIn, percLow, percHigh, isMaxRange=0):
    w, h, d = imgIn.shape
    imgOut = np.reshape(imgIn, [w * h, d]).astype(np.float64)
    mins = np.percentile(imgOut, percLow, axis=0)
#    maxs = np.percentile(imgOut, percHigh, axis=0)
    maxs = np.percentile(imgOut.flatten(), percHigh)
    difs = maxs - mins
    
    if isMaxRange==1:
      imgOut = (imgOut - mins[None, :]) / difs[None, :]
      imgOut = imgOut / difs[None, :]
    else:
#      imgOut = imgOut / maxs[None, :]
      imgOut = imgOut / maxs
      
    imgOut = np.reshape(imgOut, [w, h, d])
    print(np.max(imgOut[:,:,0].flatten()))
    imgOut = imgOut.clip(0, 1)
    imgOut = (imgOut * 255).round().astype(np.uint8)
    return imgOut


In [ ]:
def scaleImgPerc(imgIn):
    w, h, d = imgIn.shape
    imgIn = np.reshape(imgIn, [w * h, d]).astype(np.float64)
    # Get 1st and 99th percentile
    mins = np.percentile(imgIn, 1, axis=0)
    maxs = np.percentile(imgIn, 99, axis=0) - mins
    imgIn = (imgIn - mins[None, :]) / maxs[None, :]
    imgIn = np.reshape(imgIn, [w, h, d])
    imgIn = imgIn.clip(0, 1)
    return imgIn


In [ ]:
## Create a binary mask from a given polygon
def polyToMask(imgSize, polygons):
    labelMask = np.zeros(imgSize, np.uint8)
    if not polygons:
        print('    EMPTY poly object')
        return labelMask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
    cv2.fillPoly(labelMask, exteriors, 1)
    cv2.fillPoly(labelMask, interiors, 0)
    return labelMask


In [ ]:
############################################
## Hard coded paths and vars
BDIR = '/content/drive/My Drive/PrjData/Prj_DSTL/Data_Init'
IMGDIR = BDIR + '/Images_SmallSampleSize'
IMG_LIST = IMGDIR + '/ListID.csv'
  
CSV_POLY = BDIR + '/' + 'train_wkt_v4.csv'
CSV_GS = BDIR + '/' + 'grid_sizes.csv'
CSV_CTYPE = BDIR + '/' + 'DSTL_ClassTypes.csv'


In [ ]:
############################################
## Main

## Read csv files
dfP = pd.read_csv(CSV_POLY)
dfG = pd.read_csv(CSV_GS, names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
dfC = pd.read_csv(CSV_CTYPE)

dfID = pd.read_csv(IMG_LIST)


In [ ]:
dfP.head()

In [ ]:
dfG.head()

In [ ]:
dfC.head()

In [ ]:
## Read image
IMGID = '6100_2_3'

imgRGB = tiff.imread(IMGDIR + '/' + IMGID + '.tif').transpose([1, 2, 0])
imgSc = scaleImgPerc(imgRGB)
imgSc.shape

In [ ]:
tiff.imshow(imgSc)

In [ ]:
imgC = imgSc[200:1200, 200:1200,:]
tiff.imshow(imgC)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=[16,16])
ax[0,0].imshow(imgC[:,:,0], cmap='gray')
ax[0,1].imshow(imgC[:,:,1], cmap='gray')
ax[1,0].imshow(imgC[:,:,2], cmap='gray')
ax[1,1].imshow(imgC)

In [ ]:
imgCTmp = imgC[:, :, [2,1,0]]
tiff.imshow(imgCTmp)

In [ ]:
## For each image create a mask with labels of all objects
imgIds = dfID.IMG_ID.tolist()

ListImgAll = []
ListLabelAll = []
for i, tmpId in enumerate(imgIds):
#for i, tmpId in enumerate(imgIds[0:4]):

    print('\nImage no ' + str(i) + ': ' + tmpId)

    # Read grid size
    xMax = dfG[dfG.ImageId == tmpId].Xmax
    yMin = dfG[dfG.ImageId == tmpId].Ymin

    # Read RGB image, scale intensities
    imgRGB = tiff.imread(IMGDIR + '/' + tmpId + '.tif').transpose([1, 2, 0])
    #imgSc = scaleImg(imgRGB, 0, 100, 0)
    imgSc = scaleImgPerc(imgRGB)

    imgSize = imgRGB.shape[:2]
    imgX = imgSize[0]
    imgY = imgSize[1]

    # Calculate scale factors
    scX, scY = getScaleXY(imgSize, xMax, yMin)

    # Create a mask with all labels
    maskAll = np.zeros([imgX, imgY, 11], np.uint8)
    dfSel = dfP[dfP.ImageId == tmpId]
    for j, tmpRow in dfSel.iterrows():

        print(' Object no ' + str(j) + ': ' + str(tmpRow.ClassType))
        
        polyInit = shapely.wkt.loads(tmpRow.MultipolygonWKT)

        # Scale polygons
        polySc = shapely.affinity.scale(polyInit, xfact=scX, yfact=scY, origin=(0, 0, 0))
        
        maskAll[:,:,tmpRow.ClassType] = polyToMask(imgSize, polySc)
        
    # Select the object in each pixel
    maskComb = np.argmax(maskAll, axis=2).astype(np.uint8)

    # Append img and mask to out lists
    ListImgAll.append(imgSc)
    ListLabelAll.append(maskComb)



In [ ]:
dfC.Class.tolist()

In [ ]:
## Show masked area
img = ListImgAll[1]
mask = ListLabelAll[1]

cmap = matplotlib.colors.ListedColormap(['w', 'k', 'r', '#FF8C00','y','g','#FFFF00','b','c','m','#FF8EFC']) #10 classes

fig, ax = plt.subplots(1, 2, figsize=[24,24])
pImg = ax[0].imshow(img)
pMask = ax[1].imshow(mask, cmap=cmap)
cbar = fig.colorbar(pMask, ax=ax[1], fraction=0.046, pad=0.04, ticks=np.arange(0,11))
cbar.ax.set_yticklabels(dfC.Class.tolist())
cbar.ax.tick_params(labelsize=12)


In [ ]:
## Show masked area
img = ListImgAll[1][200:1200, 200:1200,:]
mask = ListLabelAll[1][200:1200, 200:1200]

cmap = matplotlib.colors.ListedColormap(['w', 'k', 'r', '#FF8C00','y','g','#FFFF00','b','c','m','#FF8EFC']) #10 classes

fig, ax = plt.subplots(1, 2, figsize=[24,24])
pImg = ax[0].imshow(img)
pMask = ax[1].imshow(mask, cmap=cmap)
cbar = fig.colorbar(pMask, ax=ax[1], fraction=0.046, pad=0.04, ticks=np.arange(0,11))
cbar.ax.set_yticklabels(dfC.Class.tolist())
cbar.ax.tick_params(labelsize=12)
